In [3]:
import pandas as pd
%matplotlib inline
import numpy as np
from scipy.special import expit
from sklearn.metrics import log_loss
import csv

### Data

In [8]:
# data_target = pd.read_csv('train/train.csv', index_col=0)
train = pd.read_pickle('train/vw_fold/train_preprocessed_lemmas_nostop.pkl')
test = pd.read_pickle('train/vw_fold/test_preprocessed_lemmas_nostop.pkl')

In [9]:
print(train.shape)
train.head()

(323430, 5)


,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,step step guide invest share market india,step step guide invest share market,0
1,3,4,story kohinoor koh noor diamond,would happen indian government steal kohinoor ...,0
2,5,6,increase speed internet connection use vpn,internet speed increase hack dns,0
3,7,8,mentally lonely solve,find remainder math 23 24 math divide 24 23,0
5,11,12,astrology capricorn sun cap moon cap rise say,triple capricorn sun moon ascendant capricorn say,1


### vw format

In [10]:
# format c tf-idf
# format usial

In [11]:
train['target'] = train['is_duplicate']
train['is_duplicate'] = train.is_duplicate.replace(0, -1)

test['target'] = test['is_duplicate']
test['is_duplicate'] = test.is_duplicate.replace(0, -1)

In [12]:
train.head()

,qid1,qid2,question1,question2,is_duplicate,target
id,,,,,,
0,1,2,step step guide invest share market india,step step guide invest share market,-1,0
1,3,4,story kohinoor koh noor diamond,would happen indian government steal kohinoor ...,-1,0
2,5,6,increase speed internet connection use vpn,internet speed increase hack dns,-1,0
3,7,8,mentally lonely solve,find remainder math 23 24 math divide 24 23,-1,0
5,11,12,astrology capricorn sun cap moon cap rise say,triple capricorn sun moon ascendant capricorn say,1,1


In [13]:
test.head()

,qid1,qid2,question1,question2,is_duplicate,target
id,,,,,,
4,9,10,one dissolve water quikly sugar salt methane c...,fish would survive salt water,-1,0
8,17,18,use シ instead し,use instead,-1,0
19,39,40,best digital marketing institution banglore,best digital marketing institute pune,-1,0
21,43,44,cause someone jealous,avoid jealous someone,-1,0
34,69,70,best travel website spain,best travel website,-1,0


In [113]:
def to_vw_format(table):
    return table.is_duplicate.astype(str) + ' |first ' + table['question1'] + ' |second ' + table['question2']

with open('vw_train.vw', 'w') as vw_train:
    vw_train.write('\n'.join(to_vw_format(train)))

with open('vw_test.vw', 'w') as vw_train:
    vw_train.write('\n'.join(to_vw_format(test)))

In [99]:
%%time
%%bash

vw -d vw_train.vw -b 25 -l 0.4 --l1 0.00000001 --l2 0.00000001 --ngram 2 -q fs -c --passes 500 --loss_function logistic -f vw_model.vw

Generating 2-grams for all namespaces.
creating quadratic features for pairs: fs 
using l1 regularization = 1e-08
using l2 regularization = 1e-08
final_regressor = vw_model.vw
Num weight bits = 25
learning rate = 0.4
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = vw_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0  -1.0000   0.0000      168
0.689781 0.686414            2            2.0  -1.0000  -0.0135      200
0.684935 0.680089            4            4.0  -1.0000  -0.0308      108
0.688288 0.691642            8            8.0  -1.0000  -0.0625      120
0.694407 0.700525           16           16.0   1.0000  -0.0226      224
0.696163 0.697920           32           32.0  -1.0000  -0.0231       32
0.689202 0.682241           64           64.0   1.

CPU times: user 8.99 ms, sys: 11.8 ms, total: 20.8 ms
Wall time: 2min 13s


In [100]:
%%bash

vw -i vw_model.vw -t -d vw_test.vw \
-p vw_test_predictions.txt

Generating 2-grams for all namespaces.
creating quadratic features for pairs: fs 
only testing
predictions = vw_test_predictions.txt
Num weight bits = 25
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = vw_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
2.833607 2.833607            1            1.0  -1.0000  -2.6833      200
1.416972 0.000336            2            2.0  -1.0000  -1.0183       32
1.918902 2.420833            4            4.0  -1.0000  -0.6305       36
10.630009 19.341116            8            8.0  -1.0000  -1.9087      160
7.925434 5.220858           16           16.0  -1.0000  -6.1741       80
4.891607 1.857781           32           32.0  -1.0000  -2.2014      192
6.380665 7.869723           64           64.0   1.0000   0.2158       60
5.874280 5.367895          128          128.0  -1.0000  -3.2099      288
5.550

### Validation

In [101]:
vw_pred = pd.read_csv('vw_test_predictions.txt', header=None, squeeze=True)

In [102]:
vw_exp_pred = expit(vw_pred)
vw_exp_pred[:10]

0    0.063964
1    0.265351
2    0.762989
3    0.347388
4    0.726593
5    0.030033
6    0.000106
7    0.129123
8    0.172313
9    0.073896
Name: 0, dtype: float64

In [103]:
log_loss(test.target.values, vw_exp_pred.values)

0.40768497612277554